# DSMarket

In [1]:
# Instalamos gdown para obtener los archivos origenes de google drive y pyarrow para guardar archivos parquet.
!pip install gdown
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 1. Importamos librerias necesarias

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import gdown


## 2. Rutas y directorios de archivos

In [3]:
sales_data_path = "https://drive.google.com/uc?id=1sbXj2mTiaHJC_1hU_blmDiU4sxy53-kN"
sales_file = "item_sales.csv"
calendar_data_path = "https://drive.google.com/uc?id=1NIqmTAKSoN3mN5MUbs-K5WXv-10srHPR"
calendar_file = "daily_calendar_with_events.csv"
prices_data_path = "https://drive.google.com/uc?id=1JHVZJ9eeqtGxPWs3vF3beEnuGVY2o8Ed"
prices_file = "item_prices.csv"
events_data_path = "https://drive.google.com/uc?id=1Mxazi67FtJcnjc0LUT8G2Ll5fu8mnLwk"
events_file = "calendar_events.csv"

## 3. Funciones globales

In [4]:
def contar_nulos(*dataframes):
    """
    Cuenta los valores nulos, la cantidad de registros por columna y el porcentaje
    de nulos para cada DataFrame proporcionado.
    """
    for i, df in enumerate(dataframes):
        print(f"\nAnálisis del DataFrame {i+1}:")
        null_counts = df.isnull().sum()
        total_counts = df.shape[0]
        null_percentage = (null_counts / total_counts) * 100

        null_info_df = pd.DataFrame({
            'Cantidad Registros': total_counts,
            'Cantidad Nulos': null_counts,
            '% Nulos': null_percentage
        })

        # Mostrar la información para cada columna
        print(null_info_df[null_info_df['Cantidad Nulos'] > 0])

In [5]:
# Funcion para asignar semana por año, primer dia de semana lunes
def weekyearnum(dt):
    return dt.strftime("%Y%W")

# Funcion para asignar fecha de semana adicionando 3 dias, primer dia miercoles
def myweekyearnum(dt):
    offsetdt = dt + timedelta(days=+2);  # you add 3 days to Mon to get to Thu
    return weekyearnum(offsetdt)

# Funcion para asignar semana por año ISO 8601, La semana empieza en lunes. La semana 1 del año es la que contiene el primer jueves del año
def weeknum(dt):
    iso = dt.isocalendar()
    return f"{iso[0]}{str(iso[1]).zfill(2)}"


## 4. Limpieza de datasets

### 4.1. Data de ventas

In [6]:
gdown.download(sales_data_path, sales_file, quiet=False)
pd_sales = pd.read_csv(sales_file)

Downloading...
From (original): https://drive.google.com/uc?id=1sbXj2mTiaHJC_1hU_blmDiU4sxy53-kN
From (redirected): https://drive.google.com/uc?id=1sbXj2mTiaHJC_1hU_blmDiU4sxy53-kN&confirm=t&uuid=1ebd51f2-d6ac-4e2e-90b0-84ad47e9d519
To: C:\Users\otalv\Documents\capstone-project-0325bcn-grupo3\02_notebooks\item_sales.csv
100%|███████████████████████████████████████████████████████████████████████████████| 121M/121M [00:12<00:00, 9.38MB/s]


In [7]:
print("Dimensiones del dataframe:", pd_sales.shape)

Dimensiones del dataframe: (30490, 1920)


In [8]:
pd_sales.head(3)

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1


In [9]:
pd_sales['store_code'].unique()

array(['NYC_1', 'NYC_2', 'NYC_3', 'NYC_4', 'BOS_1', 'BOS_2', 'BOS_3',
       'PHI_1', 'PHI_2', 'PHI_3'], dtype=object)

In [10]:
pd_sales.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Data columns (total 1920 columns):
 #     Column      Dtype 
---    ------      ----- 
 0     id          object
 1     item        object
 2     category    object
 3     department  object
 4     store       object
 5     store_code  object
 6     region      object
 7     d_1         int64 
 8     d_2         int64 
 9     d_3         int64 
 10    d_4         int64 
 11    d_5         int64 
 12    d_6         int64 
 13    d_7         int64 
 14    d_8         int64 
 15    d_9         int64 
 16    d_10        int64 
 17    d_11        int64 
 18    d_12        int64 
 19    d_13        int64 
 20    d_14        int64 
 21    d_15        int64 
 22    d_16        int64 
 23    d_17        int64 
 24    d_18        int64 
 25    d_19        int64 
 26    d_20        int64 
 27    d_21        int64 
 28    d_22        int64 
 29    d_23        int64 
 30    d_24        int64 
 31    d_25        int64 
 32  

Cantidad de **tiendas** en la data

In [11]:
print("Hay {} tiendas en la data, y el numero de registros por tienda son: ".format(pd_sales.store.nunique()))
pd_sales.store.value_counts()

Hay 10 tiendas en la data, y el numero de registros por tienda son: 


store
Greenwich_Village    3049
Harlem               3049
Tribeca              3049
Brooklyn             3049
South_End            3049
Roxbury              3049
Back_Bay             3049
Midtown_Village      3049
Yorktown             3049
Queen_Village        3049
Name: count, dtype: int64

Cantidad de **items**

In [12]:
print("Hay {} items en la data, y el numero de registros por items es: ".format(pd_sales.item.nunique()))
pd_sales.item.value_counts()

Hay 3049 items en la data, y el numero de registros por items es: 


item
ACCESORIES_1_001     10
ACCESORIES_1_002     10
ACCESORIES_1_003     10
ACCESORIES_1_004     10
ACCESORIES_1_005     10
                     ..
SUPERMARKET_3_823    10
SUPERMARKET_3_824    10
SUPERMARKET_3_825    10
SUPERMARKET_3_826    10
SUPERMARKET_3_827    10
Name: count, Length: 3049, dtype: int64

Cantidad de **departamentos**

In [13]:
print("Hay {} departmentos en la data, y el numero de registros por departamento son: ".format(pd_sales.department.nunique()))
pd_sales.department.value_counts()

Hay 7 departmentos en la data, y el numero de registros por departamento son: 


department
SUPERMARKET_3      8230
HOME_&_GARDEN_1    5320
HOME_&_GARDEN_2    5150
ACCESORIES_1       4160
SUPERMARKET_2      3980
SUPERMARKET_1      2160
ACCESORIES_2       1490
Name: count, dtype: int64

Validacion registros unicos por columna 'id'

In [14]:
pd_sales["id"].duplicated().any()

np.False_

In [15]:
pd_sales[pd_sales.id == 'ACCESORIES_1_002_NYC_1'].T

,1
id,ACCESORIES_1_002_NYC_1
item,ACCESORIES_1_002
category,ACCESORIES
department,ACCESORIES_1
store,Greenwich_Village
...,...
d_1909,1
d_1910,0
d_1911,0
d_1912,0


In [16]:
contar_nulos(pd_sales)


Análisis del DataFrame 1:
Empty DataFrame
Columns: [Cantidad Registros, Cantidad Nulos, % Nulos]
Index: []


In [17]:
pd_sales.set_index(['id','item','category','department','store','store_code','region']).head()

,,,,,,,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,item,category,department,store,store_code,region,,,,,,,,,,,,,,,,,,,,,
ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


### 4.2. Data de calendario

In [18]:
gdown.download(calendar_data_path, calendar_file, quiet=False)
pd_calendar = pd.read_csv(calendar_file)

Downloading...
From: https://drive.google.com/uc?id=1NIqmTAKSoN3mN5MUbs-K5WXv-10srHPR
To: C:\Users\otalv\Documents\capstone-project-0325bcn-grupo3\02_notebooks\daily_calendar_with_events.csv
100%|██████████████████████████████████████████████████████████████████████████████| 54.9k/54.9k [00:00<00:00, 582kB/s]


In [19]:
print("Dimensiones del dataframe:", pd_calendar.shape)

Dimensiones del dataframe: (1913, 5)


In [20]:
pd_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [21]:
pd_calendar.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [22]:
print("Rango de fechas: {} - {}".format(pd_calendar['date'].min(),pd_calendar['date'].max()))

Rango de fechas: 2011-01-29 - 2016-04-24


In [23]:
pd_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [24]:
## Asignacion de numero de semana por fecha
pd_calendar['date'] = pd.to_datetime(pd_calendar['date'], format = "%Y-%m-%d")

pd_calendar['yearweek'] = pd_calendar['date'].apply(lambda x: myweekyearnum(x))
pd_calendar['yearweek_iso'] = pd_calendar['date'].apply(lambda x: weeknum(x))

In [25]:
pd_calendar["yearweek"].unique()

array(['201105', '201106', '201107', '201108', '201109', '201110',
       '201111', '201112', '201113', '201114', '201115', '201116',
       '201117', '201118', '201119', '201120', '201121', '201122',
       '201123', '201124', '201125', '201126', '201127', '201128',
       '201129', '201130', '201131', '201132', '201133', '201134',
       '201135', '201136', '201137', '201138', '201139', '201140',
       '201141', '201142', '201143', '201144', '201145', '201146',
       '201147', '201148', '201149', '201150', '201151', '201152',
       '201200', '201201', '201202', '201203', '201204', '201205',
       '201206', '201207', '201208', '201209', '201210', '201211',
       '201212', '201213', '201214', '201215', '201216', '201217',
       '201218', '201219', '201220', '201221', '201222', '201223',
       '201224', '201225', '201226', '201227', '201228', '201229',
       '201230', '201231', '201232', '201233', '201234', '201235',
       '201236', '201237', '201238', '201239', '201240', '2012

In [26]:
contar_nulos(pd_calendar)


Análisis del DataFrame 1:
       Cantidad Registros  Cantidad Nulos    % Nulos
event                1913            1887  98.640878


Cantidad de eventos en calendario

In [27]:
pd_calendar.event.value_counts()

event
SuperBowl         6
Ramadan starts    5
Thanksgiving      5
NewYear           5
Easter            5
Name: count, dtype: int64

Agregar mas eventos al calendario, para obtener mas informacion

In [28]:
gdown.download(events_data_path, events_file, quiet=False)
pd_events = pd.read_csv(events_file)

Downloading...
From: https://drive.google.com/uc?id=1Mxazi67FtJcnjc0LUT8G2Ll5fu8mnLwk
To: C:\Users\otalv\Documents\capstone-project-0325bcn-grupo3\02_notebooks\calendar_events.csv
100%|█████████████████████████████████████████████████████████████████████████████| 2.31k/2.31k [00:00<00:00, 6.65MB/s]


In [29]:
pd_events

,date,event
0,2011-02-06,Super Bowl XLV
1,2011-02-14,Valentine's Day
2,2011-03-17,St. Patrick's Day
3,2011-04-24,Easter Sunday
4,2011-05-08,Mother's Day
...,...,...
85,2016-11-26,Small Business Saturday
86,2016-11-28,Cyber Monday
87,2016-12-12,Green Monday
88,2016-12-24,Super Saturday


In [30]:
pd_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    90 non-null     object
 1   event   90 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [31]:
pd_events['date'] = pd.to_datetime(pd_events['date'], format = "%Y-%m-%d")

In [32]:
pd_calendar = pd.merge(
    pd_calendar,
    pd_events[['date', 'event']],
    on='date',
    how='left',
    suffixes=('', '2')
)

In [33]:
pd_calendar[pd_calendar['event2'].notnull()]

,date,weekday,weekday_int,d,event,yearweek,yearweek_iso,event2
8,2011-02-06,Sunday,2,d_9,SuperBowl,201106,201105,Super Bowl XLV
16,2011-02-14,Monday,3,d_17,NaN,201107,201107,Valentine's Day
47,2011-03-17,Thursday,6,d_48,NaN,201111,201111,St. Patrick's Day
85,2011-04-24,Sunday,2,d_86,NaN,201117,201116,Easter Sunday
99,2011-05-08,Sunday,2,d_100,NaN,201119,201118,Mother's Day
...,...,...,...,...,...,...,...,...
1791,2015-12-25,Friday,7,d_1792,NaN,201551,201552,Christmas Day
1835,2016-02-07,Sunday,2,d_1836,SuperBowl,201606,201605,Super Bowl 50
1842,2016-02-14,Sunday,2,d_1843,NaN,201607,201606,Valentine's Day
1874,2016-03-17,Thursday,6,d_1875,NaN,201611,201611,St. Patrick's Day


Si la data de calendario ya contiene algun evento, mantenemos este valor, de lo contrario, usamos el valor nuevo obtenido

In [34]:
pd_calendar['final_event'] = pd_calendar['event'].fillna(pd_calendar['event2'])

In [35]:
pd_calendar[pd_calendar['final_event'].notnull()]

,date,weekday,weekday_int,d,event,yearweek,yearweek_iso,event2,final_event
8,2011-02-06,Sunday,2,d_9,SuperBowl,201106,201105,Super Bowl XLV,SuperBowl
16,2011-02-14,Monday,3,d_17,NaN,201107,201107,Valentine's Day,Valentine's Day
47,2011-03-17,Thursday,6,d_48,NaN,201111,201111,St. Patrick's Day,St. Patrick's Day
85,2011-04-24,Sunday,2,d_86,NaN,201117,201116,Easter Sunday,Easter Sunday
99,2011-05-08,Sunday,2,d_100,NaN,201119,201118,Mother's Day,Mother's Day
...,...,...,...,...,...,...,...,...,...
1798,2016-01-01,Friday,7,d_1799,NewYear,201600,201553,NaN,NewYear
1835,2016-02-07,Sunday,2,d_1836,SuperBowl,201606,201605,Super Bowl 50,SuperBowl
1842,2016-02-14,Sunday,2,d_1843,NaN,201607,201606,Valentine's Day,Valentine's Day
1874,2016-03-17,Thursday,6,d_1875,NaN,201611,201611,St. Patrick's Day,St. Patrick's Day


In [36]:
pd_calendar.drop(columns=['event', 'event2'], inplace=True)
pd_calendar.rename(columns={'final_event': 'event'}, inplace=True)

In [37]:
pd_calendar.event.value_counts()

event
SuperBowl                  6
Valentine's Day            6
St. Patrick's Day          6
Mother's Day               5
Father's Day               5
Independence Day           5
Ramadan starts             5
Halloween                  5
Labor Day                  5
Thanksgiving               5
Black Friday               5
Super Saturday             5
Small Business Saturday    5
Cyber Monday               5
Green Monday               5
NewYear                    5
Christmas Day              5
Easter                     5
Easter Sunday              1
Name: count, dtype: int64

Vemos duplicidad en uno de los eventos luego de la combinacion, procedemos a unificar sus valores.

In [38]:
pd_calendar['event'] = pd_calendar['event'].replace('Easter Sunday', 'Easter')

In [39]:
print("Hay {} semanas en la data, y el numero de registros por semana son: ".format(pd_calendar.yearweek.nunique()))
pd_calendar.yearweek.value_counts()

Hay 279 semanas en la data, y el numero de registros por semana son: 


yearweek
201616    7
201105    7
201106    7
201107    7
201108    7
         ..
201600    3
201617    2
201352    2
201200    1
201253    1
Name: count, Length: 279, dtype: int64

### 4.3. Data de precios

In [40]:
gdown.download(prices_data_path, prices_file, quiet=False)
pd_prices = pd.read_csv(prices_file)

Downloading...
From (original): https://drive.google.com/uc?id=1JHVZJ9eeqtGxPWs3vF3beEnuGVY2o8Ed
From (redirected): https://drive.google.com/uc?id=1JHVZJ9eeqtGxPWs3vF3beEnuGVY2o8Ed&confirm=t&uuid=09b7c13f-63dd-42e6-9e52-605514f7cb22
To: C:\Users\otalv\Documents\capstone-project-0325bcn-grupo3\02_notebooks\item_prices.csv
100%|███████████████████████████████████████████████████████████████████████████████| 370M/370M [00:45<00:00, 8.08MB/s]


In [41]:
print("Dimensiones del dataframe:", pd_prices.shape)

Dimensiones del dataframe: (6965706, 5)


In [42]:
pd_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [43]:
pd_prices.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


Validacion del numero de items de ventas y de precios

In [44]:
pd_prices['item'].nunique()

3049

La cantidad de items unicos es la misma, pero hay muchos registros por item. Indiica que hay precios diferenciales por item por tienda en el tiempo.

Distribucion de precios en el dataframe

In [45]:
pd_prices.describe()

,yearweek,sell_price
count,6.721786e+06,6.965706e+06
mean,2.013824e+05,5.518273e+00
std,1.450189e+02,4.387861e+00
min,2.011050e+05,1.200000e-02
25%,2.012480e+05,2.620100e+00
50%,2.014100e+05,4.200000e+00
75%,2.015150e+05,7.176000e+00
max,2.016170e+05,1.341500e+02


In [46]:
pd_prices.describe(include=['object'])

,item,category,store_code
count,6965706,6965706,6965706
unique,3049,3,10
top,HOME_&_GARDEN_2_142,SUPERMARKET,BOS_2
freq,2870,3239821,713960


In [47]:
pd_prices['yearweek'].isnull().sum()

np.int64(243920)

In [48]:
pd_prices.shape

(6965706, 5)

In [49]:
pd_prices.duplicated().sum()

np.int64(212120)

In [50]:
pd_prices = pd_prices.drop_duplicates()

In [51]:
contar_nulos(pd_prices)


Análisis del DataFrame 1:
          Cantidad Registros  Cantidad Nulos   % Nulos
yearweek             6753586           31800  0.470861


In [52]:
pd_prices.dropna(subset=['yearweek'], inplace=True)

In [53]:
pd_prices.shape

(6721786, 5)

In [54]:
pd_prices[pd_prices['item'] == "ACCESORIES_1_158"].describe()

,yearweek,sell_price
count,1233.000000,1233.000000
mean,201488.626115,29.274271
std,77.389797,1.778840
min,201327.000000,26.573400
25%,201427.000000,26.573400
50%,201509.000000,30.563400
75%,201540.000000,30.563400
max,201617.000000,30.563400


Validamos si tenemos outliers

In [55]:
# Agrupar por item y calcular min y max del precio
precio_summary = pd_prices.groupby('item')['sell_price'].agg(['min', 'max']).reset_index()

# Ordenar por precio máximo de mayor a menor
precio_summary_ordenado = precio_summary.sort_values('min', ascending=True)

print(precio_summary_ordenado)

                     item      min      max
2346    SUPERMARKET_3_122   0.0120   3.0000
2637    SUPERMARKET_3_413   0.0120   1.2000
599   HOME_&_GARDEN_1_036   0.0125   3.7250
998   HOME_&_GARDEN_1_443   0.0125   3.3000
1088  HOME_&_GARDEN_1_533   0.0125   6.2125
...                   ...      ...      ...
345      ACCESORIES_1_354  26.5734  31.8934
153      ACCESORIES_1_158  26.5734  30.5634
58       ACCESORIES_1_060  34.5800  41.2034
352      ACCESORIES_1_361  35.8834  41.2034
217      ACCESORIES_1_225  35.8834  41.2034

[3049 rows x 3 columns]


Esteblecermos umbrales inferiores y superiores para eliminar outliers de precios.

In [56]:
# 1. Calcular el promedio de sell_price por item
mean_prices = pd_prices.groupby("item")["sell_price"].mean().rename("mean_price")

# 2. Crear dataframe con umbrales superior e inferior
thresholds = (
    mean_prices
    .to_frame()
    .assign(
        upper=lambda x: x["mean_price"] * 3,
        lower=lambda x: x["mean_price"] * 0.35
    )
)

# 3. Unir con el dataframe original
pd_prices = pd_prices.merge(thresholds, on="item", how="left")

# 4. Filtrar registros que estén dentro de los umbrales
pd_prices = pd_prices[
    (pd_prices["sell_price"] <= pd_prices["upper"]) &
    (pd_prices["sell_price"] >= pd_prices["lower"])
].drop(columns=["mean_price", "upper", "lower"])


In [57]:
pd_prices[pd_prices['item'] == "ACCESORIES_2_129"].describe()

,yearweek,sell_price
count,2685.000000,2685.000000
mean,201356.252142,0.319354
std,148.514227,0.067405
min,201105.000000,0.159600
25%,201229.000000,0.305900
50%,201342.000000,0.305900
75%,201503.000000,0.305900
max,201617.000000,0.625100


### 4.4. Combinacion calendario y precios

Combinamos calendario y precios para obtener el precio por por semana-dia para cada item-store

In [58]:
pd_prices['yearweek'] = pd_prices['yearweek'].astype(int).astype(object)
pd_calendar['yearweek'] = pd_calendar['yearweek'].astype(float).astype(object)

pd_prices_day = pd.merge(pd_prices,pd_calendar[['d','yearweek', 'yearweek_iso', 'event']], how="left",
                   left_on = 'yearweek', right_on = 'yearweek')

pd_prices_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46014405 entries, 0 to 46014404
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   item          object 
 1   category      object 
 2   store_code    object 
 3   yearweek      object 
 4   sell_price    float64
 5   d             object 
 6   yearweek_iso  object 
 7   event         object 
dtypes: float64(1), object(7)
memory usage: 2.7+ GB


In [59]:
pd_prices_day

,item,category,store_code,yearweek,sell_price,d,yearweek_iso,event
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_897,201328,NaN
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_898,201328,NaN
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_899,201329,NaN
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_900,201329,NaN
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_901,201329,NaN
...,...,...,...,...,...,...,...,...
46014400,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,d_1909,201616,NaN
46014401,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,d_1910,201616,NaN
46014402,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,d_1911,201616,NaN
46014403,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201617,1.2000,d_1912,201616,NaN


In [60]:
contar_nulos(pd_prices_day)


Análisis del DataFrame 1:
       Cantidad Registros  Cantidad Nulos    % Nulos
event            46014405        43741090  95.059558


### 4.5. Anular dinamizacion de columnas de Data ventas

In [61]:
# Seleccionar solo las columnas que empiezan con 'd_'
d_columns = [col for col in pd_sales.columns if col.startswith('d_')]

# Sumar todos los valores de esas columnas
total_sales = pd_sales[d_columns].sum().sum()
print("Ventas totales antes de unpivot: ", total_sales)

Ventas totales antes de unpivot:  65695409


In [62]:
lista_dias=[]
lista_descripcion=[]
for i in pd_sales:
    if i.startswith('d_'):
      lista_dias.append(i)
    else:
      lista_descripcion.append(i)

pd_sales = pd.melt(pd_sales, id_vars=lista_descripcion, value_vars=lista_dias, var_name='d', value_name='n_sales')

In [63]:
sales_unpivot = pd_sales['n_sales'].sum()
print("Unidades totales vendidas antes de unpivot: ", total_sales)
print("Unidades totales vendidas después de unpivot: ", sales_unpivot)

Unidades totales vendidas antes de unpivot:  65695409
Unidades totales vendidas después de unpivot:  65695409


In [64]:
pd_sales.head(5)

,id,item,category,department,store,store_code,region,d,n_sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [65]:
pd_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   n_sales     int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


Agregamos el numero de semana combinando data ventas con calendario

In [66]:
pd_calendar.head(3)

,date,weekday,weekday_int,d,yearweek,yearweek_iso,event
0,2011-01-29,Saturday,1,d_1,201105.0,201104,NaN
1,2011-01-30,Sunday,2,d_2,201105.0,201104,NaN
2,2011-01-31,Monday,3,d_3,201105.0,201105,NaN


In [67]:
pd_prices_day.head(3)

,item,category,store_code,yearweek,sell_price,d,yearweek_iso,event
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_897,201328,NaN
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_898,201328,NaN
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,d_899,201329,NaN


In [68]:
pd_sales = pd_sales.merge(
    pd_calendar[["d", "date", "yearweek", "yearweek_iso", "event"]],
    on=["d"],
    how="left"
)

In [69]:
pd_sales.head()

,id,item,category,department,store,store_code,region,d,n_sales,date,yearweek,yearweek_iso,event
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN


Agregamos el precio de venta combinando data ventas con pd_prices_day

In [70]:
pd_sales= pd_sales.merge(
    pd_prices_day[["item", "store_code", "d", "sell_price"]],
    on=["item", "store_code", "d"],
    how="left"
)

Validamos que no se haya aumentado la venta despues de las combinaciones de datos.

In [71]:
sales_final = pd_sales['n_sales'].sum()
print("Unidades totales vendidas antes de unpivot: ", total_sales)
print("Unidades totales vendidas después de unpivot: ", sales_unpivot)
print("Unidades totales vendidas despues de merge: ", sales_final)

Unidades totales vendidas antes de unpivot:  65695409
Unidades totales vendidas después de unpivot:  65695409
Unidades totales vendidas despues de merge:  65695409


In [72]:
pd_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 14 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item          object        
 2   category      object        
 3   department    object        
 4   store         object        
 5   store_code    object        
 6   region        object        
 7   d             object        
 8   n_sales       int64         
 9   date          datetime64[ns]
 10  yearweek      object        
 11  yearweek_iso  object        
 12  event         object        
 13  sell_price    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(11)
memory usage: 6.1+ GB


In [73]:
pd_sales.head(3)

,id,item,category,department,store,store_code,region,d,n_sales,date,yearweek,yearweek_iso,event,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN,NaN
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,NaN,NaN


## 5. Tratamiendo de nulos

In [74]:
contar_nulos(pd_sales)


Análisis del DataFrame 1:
            Cantidad Registros  Cantidad Nulos    % Nulos
event                 58327370        55461310  95.086252
sell_price            58327370        12312965  21.110098


### 5.1. Variable 'event'

Los valores nulos en 'event' corresponden a los dias en los cuales no hay eventos marcados en calendario. Procedemos a imputar un nuevo valor de categoria.

In [75]:
pd_sales['event'] = pd_sales['event'].fillna('Without event')

### 5.2. Variable 'sell_price'

Validamos los nulos obtenidos en la columna **sell_price** para saber si existen ventas en estos dias que no tienen precio.

In [76]:
total_n_sales_nan = pd_sales[pd_sales["sell_price"].isna()]["n_sales"].sum()
print(total_n_sales_nan)

14331


No hay venta, lo cual es coherente ya que los precios semanales son un promedio del valor de ventas. Dado que en este contexto DSMarket puede establecer sus precios de venta futuros, procedemos a imputar los nulos para cada producto por tienda, interpolando los precios pasados y futuros, dejando el precio futuro encontrado o el precio pasado, lo primero que ocurra.

In [77]:
pd_sales["sell_price"] = (
    pd_sales.sort_values("yearweek")
    .groupby(["id"])["sell_price"]
    .transform(lambda s: s.interpolate(method="linear").bfill().ffill())
)

In [78]:
contar_nulos(pd_sales)


Análisis del DataFrame 1:
Empty DataFrame
Columns: [Cantidad Registros, Cantidad Nulos, % Nulos]
Index: []


Agregamos una columna calculada para el ingreso (cantidad vendida por precio de venta)

In [79]:
pd_sales["revenue"] = pd_sales["n_sales"]*pd_sales["sell_price"]

In [80]:
pd_sales.head(3)

,id,item,category,department,store,store_code,region,d,n_sales,date,yearweek,yearweek_iso,event,sell_price,revenue
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,Without event,12.7414,0.0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,Without event,5.2801,0.0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201105.0,201104,Without event,3.9501,0.0


In [81]:
pd_sales.shape

(58327370, 15)

In [83]:
# Eliminar la columna 'yearweek'
pd_sales = pd_sales.drop('yearweek', axis=1)

# Renombrar 'yearweek_iso' a 'yearweek'
pd_sales = pd_sales.rename(columns={'yearweek_iso': 'yearweek'})

## 6. Generar data para visualizacion

### 6.1. Agrupar ventas por semana, item y tienda

In [84]:
df_sales_week = (
    pd_sales
    .groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region', 'yearweek'])
    .agg(
        n_sales=('n_sales', 'sum'),
        revenue=('revenue', 'sum')
    )
    .reset_index()
)

# Ahora calculamos el promedio ponderado dividiendo
df_sales_week['avg_sell_price'] = df_sales_week['revenue'] / df_sales_week['n_sales']

In [85]:
df_sales_week

,id,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201107,0,0.0,NaN
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201108,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8354255,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,201612,17,20.4,1.2
8354256,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,201613,22,26.4,1.2
8354257,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,201614,8,9.6,1.2
8354258,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,201615,0,0.0,NaN


In [86]:
sales_week = df_sales_week['n_sales'].sum()
print("Unidades totales vendidas agrupadas: ", sales_week)

Unidades totales vendidas agrupadas:  65695409


In [87]:
# Guardar en variables cantidad unicas de semanas y de id unicos
num_weeks = df_sales_week["yearweek"].nunique()
num_ids   = df_sales_week["id"].nunique()

# Mostrar resultados
print(f"Hay {num_weeks} semanas en la data final")
print(f"Hay {num_ids} id's únicos en la data final")

# Multiplicación esperada (tamano esperado del dataframe agrupado)
expected_rows = num_weeks * num_ids
print(f"Máximo esperado de filas: {expected_rows}")

# Comparar con el shape actual
actual_rows = df_sales_week.shape[0]
print(f"Filas reales: {actual_rows}")
print(f"Diferencia: {actual_rows - expected_rows}")


Hay 274 semanas en la data final
Hay 30490 id's únicos en la data final
Máximo esperado de filas: 8354260
Filas reales: 8354260
Diferencia: 0


### 6.2. Agregar eventos a semanas agrupadas

In [88]:
df_events = (
    pd_calendar
    .dropna(subset=["event"])                     # solo semanas con evento
    .sort_values(["yearweek_iso", "date"])           # por si hay varios en la semana
    .groupby("yearweek_iso", as_index=False)
    .agg(event=("event", "first"))
)
df_events['yearweek_iso'] = df_events['yearweek_iso'].astype(int).astype(str)
df_events = df_events.rename(columns={'yearweek_iso': 'yearweek'})
df_events.head(3)

,yearweek,event
0,201105,SuperBowl
1,201107,Valentine's Day
2,201111,St. Patrick's Day


In [89]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   yearweek  81 non-null     object
 1   event     81 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [90]:
df_sales_week = df_sales_week.merge(df_events, on="yearweek", how="left")

# Rellenar semanas sin evento
df_sales_week["event"] = df_sales_week["event"].fillna("Without event")

In [91]:
df_sales_week.shape

(8354260, 12)

### 6.3. Exportar data a CSV

Guardamos la data como un archivo csv para llevar a Power BI.

In [92]:
df_sales_week.to_csv('ventas.csv')

## 7. Creacion fichero para clustering y forecasting

In [93]:
df_sales_week.to_parquet('df_sales_week.parquet', engine='pyarrow', index=False, compression='snappy')